In [1]:
import os
import json
import numpy as np
import pandas as pd
import time, logging
import gensim
from gensim.models import Phrases, KeyedVectors
from string import punctuation
import re
import unicodedata

In [2]:
!ls -lah ../data

total 4227312
drwxr-xr-x  5 rsilvei  ADESI\Domain Users   160B Jun 21 20:41 .
drwxr-xr-x  6 rsilvei  ADESI\Domain Users   192B Aug 27 13:51 ..
-rw-r--r--  1 rsilvei  ADESI\Domain Users   522M Jun 21 15:38 tweets_1.txt
-rw-r--r--  1 rsilvei  ADESI\Domain Users   341M Jun 21 16:35 tweets_2.txt
-rw-r--r--  1 rsilvei  ADESI\Domain Users   1.2G Jun 21 20:41 tweets_3.txt


In [1]:
tweet_files = [
    'tweets_1.txt',
    'tweets_2.txt',
    'tweets_3.txt'
]

In [4]:
tweet_files[0]

'tweets_1.txt'

In [5]:
tweets_data = []
for file in tweet_files:
    tweets_file = open(os.path.join('../data', file), "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue

In [6]:
print(len(tweets_data))

354640


In [7]:
for i,item in enumerate(tweets_data[0:5]):
    print(i, '-'*50)
    print(item['text'])
    print(item['lang'])
    print(item['truncated'])

0 --------------------------------------------------
RT @Anitta: 500 mil mortes... é sobre FORA BOLSONARO sim! A favor da democracia, da economia, da saúde,  da educação, do senso COLETIVO
pt
False
1 --------------------------------------------------
RT @GilDoVigor: Muitos me questionam quando a crise vai acabar e o Brasil voltar a crescer, portanto decidi responder: QUANDO O BOLSONARO E…
pt
False
2 --------------------------------------------------
@felipeneto Falar sobre politica não é dizer o que as pessoas querem ouvir e pedem não, vocês acham mesmo que vão d… https://t.co/GU2XPGIeWn
pt
True
3 --------------------------------------------------
Meu pai tomou a primeira dose da vacina '=='
pt
False
4 --------------------------------------------------
RT @depretas: umas crianças = meus filhos 🤡🤡🤡
pt
False


In [8]:
tweets = pd.DataFrame(columns=['text','lang','hashtags','user','location','truncated'])
tweets

,text,lang,hashtags,user,location,truncated


In [9]:
for x,item in enumerate(tweets_data):
    text = ""

    try:
        if item['truncated']:
            text=item['extended_tweet']['full_text']
        else:
            text=item['text']
        tweets.loc[x]=[text,item['lang'],item['entities'],item['user']['screen_name'],item['user']['location'],item['truncated']]
        if x % 5000 == 0:
            print("[{}] - TEXT - {}".format(x,text))
    except:
        pass

[0] - TEXT - RT @Anitta: 500 mil mortes... é sobre FORA BOLSONARO sim! A favor da democracia, da economia, da saúde,  da educação, do senso COLETIVO
[5000] - TEXT - RT @afroliteraria: SORTEIO RELÂMPAGO 📙✊🏾

👉🏾 Quer ganhar um dos cinco livros da imagem?

Para concorrer:
• Siga o @afroliteraria;
• Dê RT (…
[10000] - TEXT - @jairbolsonaro @minsaude Não vai falar que essa vacina é melhor que todas as outras hein.
[15000] - TEXT - RT @renatobarrosbr: Na cara não Bolsonaro, vai estragar o velório da Grobo 😂😂😂😂😂😂😂 https://t.co/nkcNr9rMX0
[20000] - TEXT - RT @Anittinhah: Anitta é tão visionária acabou de ser anunciada no conselho administrativo de um Banco. Todos os sites, mercado financeiro…
[25000] - TEXT - RT @Anitta: 500 mil mortes... é sobre FORA BOLSONARO sim! A favor da democracia, da economia, da saúde,  da educação, do senso COLETIVO
[30000] - TEXT - RT @jiiminjjkk: lembram quando o yoongi disse no vídeo do bts festa que achava que butter iria ficar por muitas semanas em #1 na hot100?

[305000] - TEXT - O Comitê de Proteção aos Jornalistas deu a história - link em inglês - da perseguição da polícia do Rio ao @demori após a denúncia feita por ele em maio https://t.co/szp3XXvw6w
[310000] - TEXT - RT @GuilhermeBoulos: A Câmara dos Deputados acaba de aprovar a privatização da Eletrobras. Enquanto o povo morre de Covid, Bolsonaro e Gued…
[315000] - TEXT - Cada vez mais eu venho acreditando que o mundo acabou em 2012 e isso aqui é o inferno. Não é possível que se Deus existe ele não tenha um pingo de vergonha de deixar o Bolsonaro respirar.
[320000] - TEXT - RT @CNNBrasil: A ministra Rosa Weber decidiu manter a decisão da CPI da Pandemia que determinou a quebra de sigilo da Associação Médicos pe…
[325000] - TEXT - O Brasil não é para amadores MESMO
[330000] - TEXT - Em defesa da democracia! Cadê a Prova Bolsonaro das fraudes na eleição que você diz ter vencido no primeiro turno? Vai com tudo TSE, pra cima dele!
[335000] - TEXT - bolsonaro vc prometeu....
[340000] - TEXT - 

In [10]:
len(tweets[tweets.lang == 'pt'])

354640

In [11]:
pt_tweets = tweets[tweets.lang == 'pt'].reset_index(drop=True)
pt_tweets.loc[19]['text']

'@qqazinha @ivetesangalo Ela sempre foi a artista que eu mais gostei no Brasil, hoje ela é só mais uma, me divirto muito com ela, mas já não é a mesma coisa, a gente vai repensando o quanto a gente pode gostar mesmo dessa galera.'

In [12]:
def remove_user(text):
    text = text.split()
    users = []
    for item in text:
        try:
            result = re.search('@(.*)', item)
            name = result.group(1)
            users.append("@"+name)
        except:
            pass
    text_filtered = [x for x in text if x not in users]
    text_filtered = " ".join(text_filtered)
    return text_filtered, users

In [13]:
def remove_latin_character(text):
    text = unicodedata.normalize('NFKD',text).encode('ASCII','ignore').decode("utf-8") # remove latin utf-8 characters
    return text

In [14]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [15]:
def remove_hashtag(text):
    text = text.split()
    hashtags = []
    for item in text:
        try:
            result = re.search('#(.*)', item)
            name = result.group(1)
            hashtags.append("#"+name)
        except:
            pass
    text_filtered = [x for x in text if x not in hashtags]
    text_filtered = " ".join(text_filtered)
    return text_filtered, hashtags

In [16]:
def remove_link(text):
    text = text.split()
    links = []
    for item in text:
        try:
            result = re.search('http(.*)', item)
            name = result.group(1)
            links.append(item)
        except:
            pass
    text_filtered = [x for x in text if x not in links]
    text_filtered = " ".join(text_filtered)
    return text_filtered, links

## Preprocessing Text

In [17]:
def preprocess(text):
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('\n', ' ')
    # Remove user
    text, _ = remove_user(text)
    # Remove hashtag
    text, _ = remove_hashtag(text)
    # Remove links 
    text, _ = remove_link(text)
    # Remove punctuation
    text = strip_punctuation(text)
    # Remove latin character
    text = remove_latin_character(text)
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    # Remove "RT" string
    text = text.replace('rt', ' ')
    #text = re.sub(r'[^\w\s]','',text) # strip punctuation
    text = ' '.join(text.split()) ## replacing extra spaces
    text = text.replace(".","")
    return text

In [18]:
pt_tweets['clean_text'] = pt_tweets['text'].apply(lambda x: preprocess(x))

In [19]:
pt_tweets.loc[19]['clean_text']

'ela sempre foi a a ista que eu mais gostei no brasil hoje ela e so mais uma me divi o muito com ela mas ja nao e a mesma coisa a gente vai repensando o quanto a gente pode gostar mesmo dessa galera'

In [20]:
for i in range(0,20):
    print(i,"-", pt_tweets['clean_text'][i])

0 - 500 mil mo es e sobre fora bolsonaro sim a favor da democracia da economia da saude da educacao do senso coletivo
1 - muitos me questionam quando a crise vai acabar e o brasil voltar a crescer po anto decidi responder quando o bolsonaro e
2 - falar sobre politica nao e dizer o que as pessoas querem ouvir e pedem nao voces acham mesmo que vao derrubar bolsonaro assim ivete apenas reforcou o que o proprio lula fala
3 - meu pai tomou a primeira dose da vacina
4 - umas criancas meus filhos
5 - anitta chama bolsonaro de louco da cabeca e diz ter votado em marina
6 - chegada de mais vacinas as vacinas serao destinadas a segunda dose dos grupos prioritarios e a complementacao da vacinacao na populacao ribeirinha
7 - e ai vai tomar a escolha e sua
8 - campina grande sendo o centro das atencoes do brasil nesse momento
9 - donos de navio que bloqueou canal de suez fazem nova proposta de compensacao
10 - ele e o maior e ponto
11 - tava aqui matutando a gente tem que pagar muito pau pra fernan

## Training W2V

In [21]:
pt_tweets.tail(5)

,text,lang,hashtags,user,location,truncated,clean_text
354635,Bolsonaro em entrevista parece todo de milícia...,pt,"{'hashtags': [{'text': 'ForaBolsonaro', 'indic...",AraujoRonair,"Mato Grosso do Sul, Brasil",False,bolsonaro em entrevista parece todo de milicia...
354636,RT @enmepaixao: Até a Beyoncé que nem é Brasil...,pt,"{'hashtags': [], 'urls': [], 'user_mentions': ...",nilodt,Brasil,False,ate a beyonce que nem e brasileira se opoe ao ...
354637,@GuilhermeBoulos Fora Bolsonaro e fora vc tmbm...,pt,"{'hashtags': [], 'urls': [], 'user_mentions': ...",kyosuketrigger,Japão,False,fora bolsonaro e fora vc tmbmbolsonaro nao e m...
354638,RT @raphaelfaraco: Vacina da Janssen pode ser ...,pt,"{'hashtags': [], 'urls': [], 'user_mentions': ...",debackerthe1,"Timbó, Brasil",False,vacina da janssen pode ser a novidade na seman...
354639,RT @guilher_rod: Fingindo que o Bolsonaro não ...,pt,"{'hashtags': [], 'urls': [], 'user_mentions': ...",isaa_alima,None,False,fingindo que o bolsonaro nao existe pra poupar...


In [22]:
raw_sentences = list(pt_tweets['clean_text'])

In [23]:
sentences = []
for sent in raw_sentences:
    sentences.append(sent.split())

In [24]:
sentences[-1]

['fingindo',
 'que',
 'o',
 'bolsonaro',
 'nao',
 'existe',
 'pra',
 'poupar',
 'minha',
 'saude',
 'mental']

In [25]:
bigram = Phrases(sentences,min_count=1)

## Training Hyperparams

In [26]:
embedded_size = 50
context_window = 8
workers=4
min_count = 1

In [27]:
model = gensim.models.Word2Vec(
        sentences = sentences,
        #sentences = bigram[sentences],
        size = embedded_size, 
        window = context_window,
        workers = workers,
        min_count = min_count,
        #sample = downsampling,
        #hs = 1,
        #negative = 0
)

In [28]:
model.wv.most_similar('gremio')

[('vermelho', 0.7961792349815369),
 ('corinthians', 0.7902805209159851),
 ('azul', 0.7893305420875549),
 ('galo', 0.778473973274231),
 ('flamengo', 0.7753100991249084),
 ('grenal', 0.7743437886238098),
 ('time', 0.7689382433891296),
 ('fluminense', 0.7503691911697388),
 ('inter', 0.7446019053459167),
 ('elenco', 0.7192239761352539)]

In [29]:
model.wv.most_similar('inter')

[('corinthians', 0.8267114758491516),
 ('nonato', 0.8125879168510437),
 ('time', 0.7991435527801514),
 ('jogador', 0.7818998694419861),
 ('bruno', 0.7675203084945679),
 ('flamengo', 0.7626039981842041),
 ('gremio', 0.7446019053459167),
 ('mendez', 0.7437172532081604),
 ('elenco', 0.7291882038116455),
 ('zagueiro', 0.72587651014328)]

In [30]:
model.wv.most_similar('politica')

[('crista', 0.5483700633049011),
 ('fama', 0.5453993678092957),
 ('etica', 0.5428282022476196),
 ('destruindoa', 0.5322074294090271),
 ('opositor', 0.5180822610855103),
 ('acomodado', 0.517890453338623),
 ('classe', 0.5128316879272461),
 ('marqueteiro', 0.5117645859718323),
 ('pratica', 0.5102018713951111),
 ('religiao', 0.5075841546058655)]

In [31]:
model.wv.most_similar('netflix')

[('tour', 0.7141731977462769),
 ('amazon', 0.6983145475387573),
 ('brp4bpnk', 0.6830480694770813),
 ('reality', 0.6782861351966858),
 ('academia', 0.6751278638839722),
 ('design', 0.6532821655273438),
 ('solteiros', 0.651692807674408),
 ('lorde', 0.6501853466033936),
 ('turne', 0.6490862369537354),
 ('loja', 0.6445334553718567)]

In [32]:
model.wv.save_word2vec_format('vetores.bin', binary=True)

In [33]:
!ls -lah

total 70504
drwxr-xr-x  8 rsilvei  ADESI\Domain Users   256B Aug 28 00:39 .
drwxr-xr-x  6 rsilvei  ADESI\Domain Users   192B Aug 27 13:51 ..
drwxr-xr-x  6 rsilvei  ADESI\Domain Users   192B Aug 27 13:53 .ipynb_checkpoints
-rw-r--r--@ 1 rsilvei  ADESI\Domain Users    40K Aug 28 00:39 1.word2vec_trainer.ipynb
-rw-r--r--@ 1 rsilvei  ADESI\Domain Users   113K Aug 27 13:45 1b.word2vec_loader.ipynb
-rw-r--r--  1 rsilvei  ADESI\Domain Users    21M Jun 22 17:20 attention_visualization.ipynb
-rw-r--r--  1 rsilvei  ADESI\Domain Users   6.5K Jun 22 17:18 gender_bias.ipynb
-rw-r--r--  1 rsilvei  ADESI\Domain Users    14M Aug 28 00:39 vetores.bin


In [34]:
model.wv.most_similar(positive=['gremio', 'inter'], negative=['cebolinha'])

[('time', 0.8338840007781982),
 ('corinthians', 0.7872154712677002),
 ('nonato', 0.7784963846206665),
 ('flamengo', 0.7773246765136719),
 ('clube', 0.7724321484565735),
 ('jogador', 0.7443618178367615),
 ('elenco', 0.7409229874610901),
 ('bruno', 0.7107099294662476),
 ('treinador', 0.7047001123428345),
 ('rival', 0.7035337090492249)]